### RL for Custom Environments
Building a shower environment to get the temoertaure right every time!

### 1. Import Dependencies 

In [1]:
# Import gym stuff
import gym
from gym import Env
from gym.spaces import Discrete,Box,Dict,Tuple,MultiBinary,MultiDiscrete

# Import helpers
import numpy as np
import random
import os

# Import stable_baselines stuff
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

### 2. Types of Spaces

In [8]:
Discrete(3).sample() # Will give a value 0,1 or 2

0

In [4]:
# Parameters : Lower and Upper value, shape of the parameter
Box(0,1,shape=(3,3)).sample() # For continuous variables or different types of sensors

array([[0.07353847, 0.8280669 , 0.905734  ],
       [0.4312524 , 0.02249796, 0.26763132],
       [0.9139449 , 0.6791878 , 0.9274099 ]], dtype=float32)

In [5]:
# Allows to combine different spaces
Tuple((Discrete(3),Box(0,1,shape=(3,))))

Tuple(Discrete(3), Box(0.0, 1.0, (3,), float32))

In [14]:
# Dictionary of different type of spaces
Dict({'height':Discrete(2),'speed':Box(0,100,shape=(1,)),"color":MultiBinary(4)}).sample()

OrderedDict([('color', array([0, 1, 1, 1], dtype=int8)),
             ('height', 1),
             ('speed', array([5.218163], dtype=float32))])

In [11]:
MultiBinary(4).sample() # Give us 4 positions

array([0, 0, 1, 0], dtype=int8)

In [13]:
MultiDiscrete([5,2,2]).sample() # 1st value will vary btw 0 and 4, 2nd and 3rd btw 0 and 1

array([4, 1, 1])

### 3. Building an environment
* Build an agent to give us the ebst shower possible.
* Randomly temperature
* 37 and 39 degrees

In [18]:
val = Discrete(3).sample()
print(val,val-1)

0 -1


In [19]:
class ShowerEnv(Env):
    def __init__(self):
        # 3 actions : whether we turn the tap up, down, unchanged
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]),high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self,action):
        # Apply action
        # 0 -1 = -1 temperature (decreasing the temp by 1 degree)
        # 1 -1 = 0
        # 2 -1 = 1 temperature (increasing the temp by 1 degree)
        self.state += action-1
        # Reduce shower length by 1 second
        self.shower_length -= 1
        # Calculate reward
        if self.state >= 37 and self.state <= 39:
            reward = 1
        else:
            reward = -1
            
        # Check if shower is done
        if self.shower_length <=0:
            done = True
        else:
            done = False
            
        # Apply temperature noise
        # self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done,info
        
    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38+random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60
        return self.state

In [15]:
Box(low=np.array([0]),high=np.array([100])).sample()

/Users/ritikagupta/anaconda3/lib/python3.11/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


array([65.90651], dtype=float32)

In [20]:
env = ShowerEnv()

/Users/ritikagupta/anaconda3/lib/python3.11/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [21]:
env.observation_space.sample()

array([70.91956], dtype=float32)

In [22]:
env.action_space.sample()

0

### 4. Test Environment

In [24]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    while not done:
        env.render()
        action = env.action_space.sample()
        obs,reward,done,info = env.step(action)
        score+= reward
    print('Episode:{} Score:{}'.format(episode,score))

Episode:1 Score:-32
Episode:2 Score:26
Episode:3 Score:-22
Episode:4 Score:-12
Episode:5 Score:-38


### 5. Train Model

In [25]:
log_path = os.path.join('`Training','Logs')
model = PPO('MlpPolicy',env,verbose=1,tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/Users/ritikagupta/anaconda3/lib/python3.11/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [28]:
model.learn(total_timesteps=50000)

Logging to `Training/Logs/PPO_3
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | 31.4     |
| time/              |          |
|    fps             | 4468     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 32.1        |
| time/                   |             |
|    fps                  | 2837        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.012513296 |
|    clip_fraction        | 0.239       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.853      |
|    explained_variance   | 0.000291    

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 34.7        |
| time/                   |             |
|    fps                  | 2212        |
|    iterations           | 11          |
|    time_elapsed         | 10          |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.010649758 |
|    clip_fraction        | 0.192       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.794      |
|    explained_variance   | 9.72e-06    |
|    learning_rate        | 0.0003      |
|    loss                 | 16.9        |
|    n_updates            | 320         |
|    policy_gradient_loss | 0.00427     |
|    value_loss           | 51.8        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60    

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 60         |
|    ep_rew_mean          | 47.7       |
| time/                   |            |
|    fps                  | 2192       |
|    iterations           | 21         |
|    time_elapsed         | 19         |
|    total_timesteps      | 43008      |
| train/                  |            |
|    approx_kl            | 0.00976998 |
|    clip_fraction        | 0.202      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.747     |
|    explained_variance   | 1.55e-06   |
|    learning_rate        | 0.0003     |
|    loss                 | 37.4       |
|    n_updates            | 420        |
|    policy_gradient_loss | 0.0112     |
|    value_loss           | 78.8       |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_m

### 6. Save Model

In [30]:
shower_path = os.path.join('Training','Saved Models','Shower_Model_PPO')

In [31]:
model.save(shower_path)

In [32]:
del model

In [33]:
model = PPO.load(shower_path,env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/Users/ritikagupta/anaconda3/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/Users/ritikagupta/anaconda3/lib/python3.11/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [34]:
evaluate_policy(model,env,n_eval_episodes=10)

/Users/ritikagupta/anaconda3/lib/python3.11/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/Users/ritikagupta/anaconda3/lib/python3.11/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(59.4, 0.9165151389911679)